# Predict with pre-trained models

This is a demo for predicting with a pre-trained model on the full imagenet dataset, which contains over 10 million images and 10 thousands classes. For a more detailed explanation, please refer to [predict.ipynb](https://github.com/dmlc/mxnet-notebooks/blob/master/python/how_to/predict.ipynb).

## Jupyter Scala kernel
Add mxnet scala jar which is created as a part of MXNet Scala package installation in classpath as follows:

**Note**: Process to add this jar in your scala kernel classpath can differ according to the scala kernel you are using.

We have used [jupyter-scala kernel](https://github.com/alexarchambault/jupyter-scala) for creating this notebook.

```
classpath.addPath(<path_to_jar>)

e.g
classpath.addPath("mxnet-full_2.11-osx-x86_64-cpu-0.1.2-SNAPSHOT.jar")
```

In [1]:
classpath.addPath("/Users/roshanin/mxnet/scala-package/assembly/osx-x86_64-cpu/target/mxnet-full_2.11-osx-x86_64-cpu-0.10.1-SNAPSHOT.jar")

Import necessary libraries:

In [2]:
import ml.dmlc.mxnet._
import ml.dmlc.mxnet.module.{FitParams, Module}
import scala.collection.immutable.ListMap
import sys.process._

import ml.dmlc.mxnet._
import ml.dmlc.mxnet.module.{FitParams, Module}
import scala.collection.immutable.ListMap
import sys.process._

Download the pretrained model as follows:

In [3]:
"wget http://data.mxnet.io/models/imagenet-11k/resnet-152/resnet-152-symbol.json -P model/ -q --show-progress"!

"wget http://data.mxnet.io/models/imagenet-11k/resnet-152/resnet-152-0000.params -P model/ -q --show-progress"!

"wget http://data.mxnet.io/models/imagenet-11k/synset.txt -P model/ -q --show-progress"!


     0K .......... .......... .......... .......... .......... 22%  121K 1s
    50K .......... .......... .......... .......... .......... 45%  157K 1s
   100K .......... .......... .......... .......... .......... 68%  205K 0s
   150K .......... .......... .......... .......... .......... 91%  297K 0s
   200K .......... ........                                   100%  121K=1.3s

res2_0: Int = 0
res2_1: Int = 0
res2_2: Int = 0

## Initialization
We first load the model into memory with loadCheckpoint. It returns the symbol (see [symbol_scala.ipynb](https://github.com/dmlc/mxnet-notebooks/blob/master/scala/basic/symbol_scala.ipynb)) definition of the neural network, and parameters.

In [4]:
val (resnet, argParamsResnet, auxParamsResnet) = Model.loadCheckpoint("model/resnet-152", 0)

log4j:WARN No appenders could be found for logger (MXNetJVM).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


resnet: Symbol = ml.dmlc.mxnet.Symbol@7ce32cb
argParamsResnet: Map[String, NDArray] = Map(
  "stage3_unit14_bn2_gamma" -> ml.dmlc.mxnet.NDArray@c296fced,
  "stage3_unit17_conv1_weight" -> ml.dmlc.mxnet.NDArray@75334891,
  "stage3_unit8_conv3_weight" -> ml.dmlc.mxnet.NDArray@783b9ecd,
  "stage2_unit7_bn3_beta" -> ml.dmlc.mxnet.NDArray@eeb61ac7,
  "stage1_unit3_conv3_weight" -> ml.dmlc.mxnet.NDArray@b5c72854,
  "stage3_unit14_bn1_beta" -> ml.dmlc.mxnet.NDArray@8c34f84f,
  "stage3_unit10_bn1_beta" -> ml.dmlc.mxnet.NDArray@b2d0a2b8,
  "stage3_unit2_bn1_gamma" -> ml.dmlc.mxnet.NDArray@40533d09,
  "stage3_unit18_conv1_weight" -> ml.dmlc.mxnet.NDArray@285d08cd,
  "stage3_unit20_bn1_gamma" -> ml.dmlc.mxnet.NDArray@53a87a2c,
  "stage3_unit29_bn3_beta" -> ml.dmlc.mxnet.NDArray@957e96f2,
  "stage2_unit3_conv2_weight" -> ml.dmlc.mxnet.NDArray@d08559ca,
  "stage3_unit25_bn2_gamma" -> ml.dmlc.mxnet.NDArray@c8bc6afe,
  "stage4_unit3_bn1_beta" -> ml.dmlc.mxnet.NDArray@2d738385,
  "stage1_unit3_bn1_bet

Create a model for this model on GPU 0.


In [5]:
val mod = new Module(resnet, contexts = Context.cpu())
val dataShapesResnet = ListMap("data" -> Shape(1, 3, 224, 224))
mod.bind(dataShapes=dataShapesResnet, forTraining = false)
mod.setParams(argParamsResnet, auxParamsResnet)

mod: Module = ml.dmlc.mxnet.module.Module@d76fa60
dataShapesResnet: ListMap[String, Shape] = Map("data" -> (1,3,224,224))

Next we define the function to obtain an image by a given URL and the function for predicting.


In [6]:
classpath.addPath("/usr/local/opt/opencv3/share/OpenCV/java/opencv-320.jar")

In [7]:
import org.opencv.core.Core
//import org.opencv.highgui.Highgui
import org.opencv.imgcodecs.Imgcodecs
import org.opencv.imgproc.Imgproc
import ml.dmlc.mxnet.NDArray
import org.opencv.core.Mat
import org.opencv.core.CvType
import java.util.ArrayList
import org.opencv.core.Size

import org.opencv.core.Core
import org.opencv.imgcodecs.Imgcodecs
import org.opencv.imgproc.Imgproc
import ml.dmlc.mxnet.NDArray
import org.opencv.core.Mat
import org.opencv.core.CvType
import java.util.ArrayList
import org.opencv.core.Size

In [8]:
def getImage(filename: String) ={
    System.loadLibrary(Core.NATIVE_LIBRARY_NAME);
    println(String.format("Loaded OpenCV %s", Core.VERSION))
    println(filename)
    val mat = Imgcodecs.imread(filename)
    val greyMat = new Mat()
    Imgproc.cvtColor(mat, greyMat, Imgproc.COLOR_BGR2RGB)
    val resizeMat = new Mat()
    Imgproc.resize(mat, resizeMat, new Size(224, 224))
    
    val typeMat = new Mat()
    resizeMat.convertTo(typeMat, CvType.CV_32F)
    
    val size = (typeMat.total * typeMat.channels).toInt
    val buff = new Array[Float](size)
    typeMat.get(0, 0, buff)
    
    println(buff)
    typeMat
}

defined function getImage

In [10]:
val img = getImage("0.jpg")

Loaded OpenCV 3.2.0
0.jpg
[F@4a8d4eba


img: Mat = Mat [ 224*224*CV_32FC3, isCont=true, isSubmat=false, nativeObj=0x7f98f89efb50, dataAddr=0x16c766020 ]